# Forest Restoration and Reforestation Analysis

## 🧭 01 – Data Preparation and Cleaning

This notebook represents the first major phase of the data science workflow for the project AI for Sustainable Forest Restoration and Reforestation Analysis, developed within the Introduction to Data Science course of the Erasmus Mundus Joint Master’s Programme Artificial Intelligence for Sustainable Societies (AISS).

The goal of this stage is to load, explore, clean, and merge multiple open-access datasets from Global Forest Watch (GFW) into a single, well-structured analytical dataset that can later be used for exploratory data analysis (EDA), machine-learning modeling, and interactive visualization through a Dash web dashboard.

## 🎯 Objectives

Ingest raw data from the GFW Excel workbook containing nine sheets at both country- and subnational-levels.

Filter, normalize, and restructure the relevant country-level sheets:

Country tree cover loss

Country primary loss

Country drivers

Country carbon data

Transform the data into a tidy format by unpivoting year-based columns, harmonizing thresholds, and renaming variables for clarity.

Integrate multiple datasets (loss, primary loss, drivers, and carbon emissions) into one comprehensive dataframe.

Validate data quality through summary statistics and visual inspections.

Export the cleaned dataset (merged_clean_data.csv) to the data/processed/ directory for use in downstream analysis and the Dash dashboard.

## 🌍 Context

Deforestation and reforestation patterns vary significantly across regions, and the integration of diverse GFW datasets provides a quantitative foundation to evaluate how forest restoration progress contributes to Sustainable Development Goal 15 – Life on Land.
By preparing and standardizing these data systematically, this notebook ensures that subsequent analytical phases—EDA, modeling, and dashboard development—are accurate, reproducible, and ready for advanced AI-based insights.

## Questions to be answered
1. how much tree cover exists?
2. how much is lost?
3. what causes the loss?
4. how that affects carbon emissions and climate?

### 🧩 Step 0 — Setup & Imports

In [1]:
import pandas as pd

RAW_PATH = "../data/raw/global_forest_watch_raw_data.xlsx"

# Read Excel workbook
global_forest_watch_excel_file = pd.ExcelFile(RAW_PATH)
global_forest_watch_excel_file.sheet_names


['Read_Me',
 'Country tree cover loss',
 'Country primary loss',
 'Country drivers',
 'Country carbon data',
 'Subnational 1 tree cover loss',
 'Subnational 1 primary loss',
 'Subnational 1 drivers',
 'Subnational 1 carbon data']

### 🌲 1. Processing the “Country Tree Cover Loss” Sheet

This dataset provides annual tree cover loss (in hectares) per country and canopy threshold between 2001–2024.
It represents total forest area lost, regardless of forest type or cause.

Our main goals for this section are to:
- Audit the raw dataset to understand structure, missingness, and data types.
- Clean the data by standardizing column names, fixing country names, and converting data types.
- Reshape wide-format yearly columns (tc_loss_ha_2001, tc_loss_ha_2002, …) into a tidy long format with a single year column.
- Save the processed output to data/processed/country_tree_cover_loss_processed.csv.

🧩 STEP 1 — Raw Data Audit

🧠 What this does:
- Loads the sheet safely (no edits).
- Prints structure, data types, and missing values.
- Checks for duplicates and previews a few rows.

In [2]:
import pandas as pd
RAW_PATH = "../data/raw/global_forest_watch_raw_data.xlsx"
global_forest_watch_excel_file = pd.ExcelFile(RAW_PATH)
country_tree_cover_loss_sheet = global_forest_watch_excel_file.parse("Country tree cover loss")
print(country_tree_cover_loss_sheet.info())
print(country_tree_cover_loss_sheet.describe)
print(country_tree_cover_loss_sheet['country'].nunique())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1328 entries, 0 to 1327
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   country            1328 non-null   object
 1   threshold          1328 non-null   int64 
 2   area_ha            1328 non-null   int64 
 3   extent_2000_ha     1328 non-null   int64 
 4   extent_2010_ha     1328 non-null   int64 
 5   gain_2000-2012_ha  1328 non-null   int64 
 6   tc_loss_ha_2001    1328 non-null   int64 
 7   tc_loss_ha_2002    1328 non-null   int64 
 8   tc_loss_ha_2003    1328 non-null   int64 
 9   tc_loss_ha_2004    1328 non-null   int64 
 10  tc_loss_ha_2005    1328 non-null   int64 
 11  tc_loss_ha_2006    1328 non-null   int64 
 12  tc_loss_ha_2007    1328 non-null   int64 
 13  tc_loss_ha_2008    1328 non-null   int64 
 14  tc_loss_ha_2009    1328 non-null   int64 
 15  tc_loss_ha_2010    1328 non-null   int64 
 16  tc_loss_ha_2011    1328 non-null   int64 


In [38]:
import numpy as np

# --- Load raw sheet (read-only) ---
country_tree_cover_loss_sheet = global_forest_watch_excel_file.parse("Country tree cover loss")

print("Loaded 'Country tree cover loss' sheet")
print("Shape (rows, columns):", country_tree_cover_loss_sheet.shape)

print()
# 1️⃣ Basic info — data types & non-null counts
print("\n Basic Info:")
country_tree_cover_loss_sheet.info()

# 2️⃣ Missing values summary
print("\n🔹 Missing values per column:")
display(
    country_tree_cover_loss_sheet.isna()
    .sum()
    .to_frame("missing_count")
    .sort_values("missing_count", ascending=False)
    .head(10)
)

# 3️⃣ Numeric summary (detect anomalies)
print("\n🔹 Descriptive statistics (numeric columns):")
display(country_tree_cover_loss_sheet.describe(include=[np.number]).T.head(10))

# 4️⃣ Duplicate check for country-threshold pairs
dup_count = country_tree_cover_loss_sheet.duplicated(subset=["country", "threshold"]).sum()
print(f"\n🔹 Duplicate country–threshold pairs: {dup_count}")

# 5️⃣ Sample rows
print("\n🔹 Sample preview (first 5 rows):")
display(country_tree_cover_loss_sheet.head(5))


✅ Loaded 'Country tree cover loss' sheet
Shape (rows, columns): (1328, 30)

🔹 Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1328 entries, 0 to 1327
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   country            1328 non-null   object
 1   threshold          1328 non-null   int64 
 2   area_ha            1328 non-null   int64 
 3   extent_2000_ha     1328 non-null   int64 
 4   extent_2010_ha     1328 non-null   int64 
 5   gain_2000-2012_ha  1328 non-null   int64 
 6   tc_loss_ha_2001    1328 non-null   int64 
 7   tc_loss_ha_2002    1328 non-null   int64 
 8   tc_loss_ha_2003    1328 non-null   int64 
 9   tc_loss_ha_2004    1328 non-null   int64 
 10  tc_loss_ha_2005    1328 non-null   int64 
 11  tc_loss_ha_2006    1328 non-null   int64 
 12  tc_loss_ha_2007    1328 non-null   int64 
 13  tc_loss_ha_2008    1328 non-null   int64 
 14  tc_loss_ha_2009    1328 non-null   int64 
 15 

,missing_count
country,0
threshold,0
tc_loss_ha_2023,0
tc_loss_ha_2022,0
tc_loss_ha_2021,0
tc_loss_ha_2020,0
tc_loss_ha_2019,0
tc_loss_ha_2018,0
tc_loss_ha_2017,0
tc_loss_ha_2016,0



🔹 Descriptive statistics (numeric columns):


,count,mean,std,min,25%,50%,75%,max
threshold,1328.0,2.812500e+01,2.249979e+01,0.0,13.75,22.5,35.00,7.500000e+01
area_ha,1328.0,7.814805e+07,2.015752e+08,2094.0,5117777.00,20225865.5,62019967.00,1.689455e+09
extent_2000_ha,1328.0,3.038020e+07,1.056704e+08,0.0,548025.50,3622985.5,18319767.50,1.689455e+09
extent_2010_ha,1328.0,2.994347e+07,1.047382e+08,0.0,541270.00,3499126.0,18198855.25,1.689455e+09
gain_2000-2012_ha,1328.0,7.867308e+05,3.417373e+06,0.0,13832.00,94359.0,388240.00,3.722054e+07
tc_loss_ha_2001,1328.0,7.906576e+04,3.124548e+05,0.0,514.50,6940.5,31545.25,2.933201e+06
tc_loss_ha_2002,1328.0,9.644243e+04,4.116669e+05,0.0,393.25,5172.0,32422.25,3.715945e+06
tc_loss_ha_2003,1328.0,8.478851e+04,3.768918e+05,0.0,312.00,3940.5,28489.25,3.489258e+06
tc_loss_ha_2004,1328.0,1.162822e+05,4.947177e+05,0.0,532.75,6147.5,38602.75,4.133606e+06
tc_loss_ha_2005,1328.0,1.068036e+05,4.231545e+05,0.0,548.25,7207.5,40362.50,3.675951e+06



🔹 Duplicate country–threshold pairs: 0

🔹 Sample preview (first 5 rows):


,country,threshold,area_ha,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tc_loss_ha_2001,tc_loss_ha_2002,tc_loss_ha_2003,tc_loss_ha_2004,...,tc_loss_ha_2015,tc_loss_ha_2016,tc_loss_ha_2017,tc_loss_ha_2018,tc_loss_ha_2019,tc_loss_ha_2020,tc_loss_ha_2021,tc_loss_ha_2022,tc_loss_ha_2023,tc_loss_ha_2024
0,Afghanistan,0,64383655,64383655,64383655,10738,103,214,267,226,...,0,0,0,31,25,46,47,16,133,223
1,Afghanistan,10,64383655,432070,126231,10738,92,190,254,207,...,0,0,0,28,19,40,37,9,32,32
2,Afghanistan,15,64383655,302629,106852,10738,91,186,248,205,...,0,0,0,28,19,39,32,7,23,17
3,Afghanistan,20,64383655,284330,105718,10738,89,181,245,203,...,0,0,0,28,18,39,32,7,22,16
4,Afghanistan,25,64383655,254843,72384,10738,89,180,244,202,...,0,0,0,27,18,38,27,6,21,14


🧩 STEP 1 — Cleaning “Country tree cover loss”

🎯 Goal
- Detect and handle small issues before transforming.
- Ensure the data has consistent country names, numeric types, and no duplicate rows.

In [12]:
# Work on a copy to avoid touching raw data
tcl_clean = country_tree_cover_loss_sheet.copy()

# --- 1️⃣ Standardize column names ---
tcl_clean.columns = tcl_clean.columns.str.strip().str.lower().str.replace(" ", "_")

# --- 2️⃣ Clean country names ---
if "country" in tcl_clean.columns:
    tcl_clean["country"] = tcl_clean["country"].astype(str).str.strip().str.title()

# --- 3️⃣ Replace empty strings with NaN ---
tcl_clean = tcl_clean.replace(r"^\s*$", pd.NA, regex=True)

# --- 4️⃣ Remove duplicate rows (if any) ---
before = len(tcl_clean)
tcl_clean = tcl_clean.drop_duplicates(subset=["country", "threshold"], keep="first")
after = len(tcl_clean)
print(f"Removed {before - after} duplicate rows (if any).")

# --- 5️⃣ Convert numeric columns properly ---
numeric_cols = tcl_clean.select_dtypes(include="object").columns
for col in numeric_cols:
    try:
        tcl_clean[col] = pd.to_numeric(tcl_clean[col])
    except (ValueError, TypeError):
        #keep it as-is
        pass

# --- 6️⃣ Verify results ---
print("\n✅ After cleaning:")
display(tcl_clean.info())
display(tcl_clean.head(5))


Removed 0 duplicate rows (if any).

✅ After cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1328 entries, 0 to 1327
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   country            1328 non-null   object
 1   threshold          1328 non-null   int64 
 2   area_ha            1328 non-null   int64 
 3   extent_2000_ha     1328 non-null   int64 
 4   extent_2010_ha     1328 non-null   int64 
 5   gain_2000-2012_ha  1328 non-null   int64 
 6   tc_loss_ha_2001    1328 non-null   int64 
 7   tc_loss_ha_2002    1328 non-null   int64 
 8   tc_loss_ha_2003    1328 non-null   int64 
 9   tc_loss_ha_2004    1328 non-null   int64 
 10  tc_loss_ha_2005    1328 non-null   int64 
 11  tc_loss_ha_2006    1328 non-null   int64 
 12  tc_loss_ha_2007    1328 non-null   int64 
 13  tc_loss_ha_2008    1328 non-null   int64 
 14  tc_loss_ha_2009    1328 non-null   int64 
 15  tc_loss_ha_2010    1328 non-null   

None

,country,threshold,area_ha,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tc_loss_ha_2001,tc_loss_ha_2002,tc_loss_ha_2003,tc_loss_ha_2004,...,tc_loss_ha_2015,tc_loss_ha_2016,tc_loss_ha_2017,tc_loss_ha_2018,tc_loss_ha_2019,tc_loss_ha_2020,tc_loss_ha_2021,tc_loss_ha_2022,tc_loss_ha_2023,tc_loss_ha_2024
0,Afghanistan,0,64383655,64383655,64383655,10738,103,214,267,226,...,0,0,0,31,25,46,47,16,133,223
1,Afghanistan,10,64383655,432070,126231,10738,92,190,254,207,...,0,0,0,28,19,40,37,9,32,32
2,Afghanistan,15,64383655,302629,106852,10738,91,186,248,205,...,0,0,0,28,19,39,32,7,23,17
3,Afghanistan,20,64383655,284330,105718,10738,89,181,245,203,...,0,0,0,28,18,39,32,7,22,16
4,Afghanistan,25,64383655,254843,72384,10738,89,180,244,202,...,0,0,0,27,18,38,27,6,21,14


🧩 STEP 3 — Transform “Country tree cover loss” to Long Format

🧠 Why we’re doing this

- Makes analysis and visualization possible (e.g., plotting tree loss over time).
- Converts ~24 year columns into one year column + one tree_cover_loss_ha column.
- Keeps country, threshold, and other metadata intact.

In [13]:
def melt_yearly_columns(df, prefix, value_name):
    """
    Converts wide year columns (e.g., tc_loss_ha_2001, tc_loss_ha_2002, …).
    """
    # Detect all columns that start with the prefix
    year_cols = [c for c in df.columns if c.startswith(prefix)]

    if not year_cols:
        print(f"No columns found with prefix '{prefix}'. Check column names.")
        return df

    melted = df.melt(
        id_vars=[c for c in df.columns if c not in year_cols],
        value_vars=year_cols,
        var_name="metric_year",
        value_name=value_name
    )
    # Extract year as integer from the column name
    melted["year"] = melted["metric_year"].str.extract(r"(\d{4})").astype(int)

    # Drop the temporary column
    melted = melted.drop(columns=["metric_year"])

    return melted
# --- Apply transformation ---
tcl_tidy = melt_yearly_columns(tcl_clean, prefix="tc_loss_ha_", value_name="tree_cover_loss_ha")

print("✅ Transformed shape:", tcl_tidy.shape)
print("Columns:", list(tcl_tidy.columns)[:10])

print("\n📊 Preview of tidy data (AFTER processing):")
display(tcl_tidy.head(10))


✅ Transformed shape: (31872, 8)
Columns: ['country', 'threshold', 'area_ha', 'extent_2000_ha', 'extent_2010_ha', 'gain_2000-2012_ha', 'tree_cover_loss_ha', 'year']

📊 Preview of tidy data (AFTER processing):


,country,threshold,area_ha,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tree_cover_loss_ha,year
0,Afghanistan,0,64383655,64383655,64383655,10738,103,2001
1,Afghanistan,10,64383655,432070,126231,10738,92,2001
2,Afghanistan,15,64383655,302629,106852,10738,91,2001
3,Afghanistan,20,64383655,284330,105718,10738,89,2001
4,Afghanistan,25,64383655,254843,72384,10738,89,2001
5,Afghanistan,30,64383655,205771,71786,10738,88,2001
6,Afghanistan,50,64383655,148417,46235,10738,78,2001
7,Afghanistan,75,64383655,75480,18268,10738,46,2001
8,Albania,0,2872761,2872761,2872761,16468,3907,2001
9,Albania,10,2872761,838601,712542,16468,3815,2001


💾 STEP 4 — Save the Processed Data

In [15]:
import os

# Create folder if not already present
os.makedirs("../data/processed", exist_ok=True)

# Define output path
tcl_out_path = "../data/processed/country_tree_cover_loss_processed.csv"

# Save the processed tidy data
tcl_tidy.to_csv(tcl_out_path, index=False)

print(f"💾 Saved processed dataset to: {tcl_out_path}")
print(f"Rows: {len(tcl_tidy):,} | Columns: {len(tcl_tidy.columns)}")

# Quick verification: reload and confirm structure
verify = pd.read_csv(tcl_out_path)
print("\n✅ Reloaded successfully! Sample below:")
display(verify.head(10))


💾 Saved processed dataset to: ../data/processed/country_tree_cover_loss_processed.csv
Rows: 31,872 | Columns: 8

✅ Reloaded successfully! Sample below:


,country,threshold,area_ha,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tree_cover_loss_ha,year
0,Afghanistan,0,64383655,64383655,64383655,10738,103,2001
1,Afghanistan,10,64383655,432070,126231,10738,92,2001
2,Afghanistan,15,64383655,302629,106852,10738,91,2001
3,Afghanistan,20,64383655,284330,105718,10738,89,2001
4,Afghanistan,25,64383655,254843,72384,10738,89,2001
...,...,...,...,...,...,...,...,...
95,Belize,75,2204340,1575027,1575027,18728,7866,2001
96,Benin,0,11528631,11528631,11528631,181245,18084,2001
97,Benin,10,11528631,8877131,7726250,181245,17778,2001
98,Benin,15,11528631,4559877,4251239,181245,10302,2001


🌴 2. Processing the “Country Primary Loss” Sheet

This dataset focuses on humid tropical primary forests, providing annual primary forest loss (in hectares) for 2002–2024.
It reflects the most ecologically significant areas of forest change.

Our main goals for this section are to:

- Audit the raw dataset to check column structure and detect anomalies.
- Clean column names (notably fixing area__ha → area_ha) and standardize country names.
- Reshape the yearly loss columns (tc_loss_ha_2002, tc_loss_ha_2003, …) into a tidy long format.

Save the processed output to data/processed/country_primary_loss_processed.csv.

🌴 STEP 1 — Audit “Country primary loss”

🧠 Why we audit again

Even though the structure is similar, this sheet:
- Focuses specifically on humid tropical primary forests (2002–2024).
- May have slightly different column names (for example area__ha instead of area_ha).
- Sometimes starts a year later (2002) and may have unique missingness patterns.

In [16]:
# --- Load the sheet ---
pl_raw = global_forest_watch_excel_file.parse("Country primary loss")

print("✅ Loaded 'Country primary loss' sheet")
print("Shape (rows, columns):", pl_raw.shape)

# 1️⃣ Basic info
print("\n🔹 Basic Info:")
pl_raw.info()

# 2️⃣ Missing values summary
print("\n🔹 Missing values per column:")
display(
    pl_raw.isna()
    .sum()
    .to_frame("missing_count")
    .sort_values("missing_count", ascending=False)
    .head(10)
)

# 3️⃣ Numeric summary
print("\n🔹 Descriptive statistics (numeric columns):")
display(pl_raw.describe(include=[np.number]).T.head(10))

# 4️⃣ Duplicate check
dup_count = pl_raw.duplicated(subset=["country", "threshold"]).sum()
print(f"\n🔹 Duplicate country–threshold pairs: {dup_count}")

# 5️⃣ Sample rows
print("\n🔹 Sample of raw data:")
display(pl_raw.head(5))


✅ Loaded 'Country primary loss' sheet
Shape (rows, columns): (76, 26)

🔹 Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   country          76 non-null     object
 1   threshold        76 non-null     int64 
 2   area__ha         76 non-null     int64 
 3   tc_loss_ha_2002  76 non-null     int64 
 4   tc_loss_ha_2003  76 non-null     int64 
 5   tc_loss_ha_2004  76 non-null     int64 
 6   tc_loss_ha_2005  76 non-null     int64 
 7   tc_loss_ha_2006  76 non-null     int64 
 8   tc_loss_ha_2007  76 non-null     int64 
 9   tc_loss_ha_2008  76 non-null     int64 
 10  tc_loss_ha_2009  76 non-null     int64 
 11  tc_loss_ha_2010  76 non-null     int64 
 12  tc_loss_ha_2011  76 non-null     int64 
 13  tc_loss_ha_2012  76 non-null     int64 
 14  tc_loss_ha_2013  76 non-null     int64 
 15  tc_loss_ha_2014  76 non-null     int64 
 1

,missing_count
country,0
threshold,0
tc_loss_ha_2023,0
tc_loss_ha_2022,0
tc_loss_ha_2021,0
tc_loss_ha_2020,0
tc_loss_ha_2019,0
tc_loss_ha_2018,0
tc_loss_ha_2017,0
tc_loss_ha_2016,0



🔹 Descriptive statistics (numeric columns):


,count,mean,std,min,25%,50%,75%,max
threshold,76.0,3.000000e+01,0.000000e+00,30.0,30.00,30.0,30.00,30.0
area__ha,76.0,1.349796e+07,4.300172e+07,1653.0,227151.25,1833099.5,7487047.00,343260979.0
tc_loss_ha_2002,76.0,3.500986e+04,1.883565e+05,0.0,132.50,2029.5,10132.50,1621738.0
tc_loss_ha_2003,76.0,3.268253e+04,1.816783e+05,0.0,122.50,1582.5,10143.75,1570540.0
tc_loss_ha_2004,76.0,4.462941e+04,2.367232e+05,0.0,203.50,2277.0,11972.75,2016350.0
tc_loss_ha_2005,76.0,4.370550e+04,2.157229e+05,0.0,217.00,2320.0,12021.00,1824217.0
tc_loss_ha_2006,76.0,3.695367e+04,1.706006e+05,0.0,245.50,2528.5,14184.50,1415536.0
tc_loss_ha_2007,76.0,3.814787e+04,1.452392e+05,0.0,184.50,3063.5,18432.00,1149515.0
tc_loss_ha_2008,76.0,3.563292e+04,1.354756e+05,0.0,222.75,3677.5,15289.25,1075087.0
tc_loss_ha_2009,76.0,3.676518e+04,1.160801e+05,0.0,445.25,3729.5,19996.25,700115.0



🔹 Duplicate country–threshold pairs: 0

🔹 Sample of raw data:


,country,threshold,area__ha,tc_loss_ha_2002,tc_loss_ha_2003,tc_loss_ha_2004,tc_loss_ha_2005,tc_loss_ha_2006,tc_loss_ha_2007,tc_loss_ha_2008,...,tc_loss_ha_2015,tc_loss_ha_2016,tc_loss_ha_2017,tc_loss_ha_2018,tc_loss_ha_2019,tc_loss_ha_2020,tc_loss_ha_2021,tc_loss_ha_2022,tc_loss_ha_2023,tc_loss_ha_2024
0,Angola,30,2458061,3499,2963,2354,3110,1400,8060,2699,...,8998,12040,11166,13507,9995,8895,24326,15576,17627,13660
1,Argentina,30,4418724,9318,14459,28090,31429,24095,18687,47067,...,10547,15247,17202,9496,8983,20847,11921,21388,11473,12103
2,Australia,30,13977,0,0,0,0,25,0,0,...,5,0,0,0,5,0,0,0,0,0
3,Bangladesh,30,101114,619,266,347,306,677,369,240,...,205,345,414,358,387,459,308,307,743,467
4,Belize,30,1165487,5570,2993,2108,3206,1899,4140,3632,...,6606,11511,6616,4781,8772,16087,4560,4033,11667,21137


🧩 STEP 2 — Cleaning Country primary loss
🧠 What this does
- Fixes area__ha naming.
- Standardizes and cleans country names.
- Removes duplicates.

In [17]:
# Work on a copy
pl_clean = pl_raw.copy()

# --- 1️⃣ Standardize column names ---
pl_clean.columns = pl_clean.columns.str.strip().str.lower().str.replace(" ", "_")

# --- 2️⃣ Rename inconsistent columns ---
if "area__ha" in pl_clean.columns:
    pl_clean = pl_clean.rename(columns={"area__ha": "area_ha"})
    print("Renamed column 'area__ha' → 'area_ha'")

# --- 3️⃣ Clean country names ---
if "country" in pl_clean.columns:
    pl_clean["country"] = pl_clean["country"].astype(str).str.strip().str.title()

# --- 4️⃣ Replace empty strings with NaN ---
pl_clean = pl_clean.replace(r"^\s*$", pd.NA, regex=True)

# --- 5️⃣ Drop duplicates ---
before = len(pl_clean)
pl_clean = pl_clean.drop_duplicates(subset=["country", "threshold"], keep="first")
after = len(pl_clean)
print(f"Removed {before - after} duplicate rows (if any).")

# --- 6️⃣ Convert numeric-like columns ---
for col in pl_clean.columns:
    try:
        pl_clean[col] = pd.to_numeric(pl_clean[col])
    except (ValueError, TypeError):
        pass

# --- 7️⃣ Verify results ---
print("\n✅ After cleaning:")
display(pl_clean.info())
display(pl_clean.head(5))


Renamed column 'area__ha' → 'area_ha'
Removed 0 duplicate rows (if any).

✅ After cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   country          76 non-null     object
 1   threshold        76 non-null     int64 
 2   area_ha          76 non-null     int64 
 3   tc_loss_ha_2002  76 non-null     int64 
 4   tc_loss_ha_2003  76 non-null     int64 
 5   tc_loss_ha_2004  76 non-null     int64 
 6   tc_loss_ha_2005  76 non-null     int64 
 7   tc_loss_ha_2006  76 non-null     int64 
 8   tc_loss_ha_2007  76 non-null     int64 
 9   tc_loss_ha_2008  76 non-null     int64 
 10  tc_loss_ha_2009  76 non-null     int64 
 11  tc_loss_ha_2010  76 non-null     int64 
 12  tc_loss_ha_2011  76 non-null     int64 
 13  tc_loss_ha_2012  76 non-null     int64 
 14  tc_loss_ha_2013  76 non-null     int64 
 15  tc_loss_ha_2014  76 non-null     in

None

,country,threshold,area_ha,tc_loss_ha_2002,tc_loss_ha_2003,tc_loss_ha_2004,tc_loss_ha_2005,tc_loss_ha_2006,tc_loss_ha_2007,tc_loss_ha_2008,...,tc_loss_ha_2015,tc_loss_ha_2016,tc_loss_ha_2017,tc_loss_ha_2018,tc_loss_ha_2019,tc_loss_ha_2020,tc_loss_ha_2021,tc_loss_ha_2022,tc_loss_ha_2023,tc_loss_ha_2024
0,Angola,30,2458061,3499,2963,2354,3110,1400,8060,2699,...,8998,12040,11166,13507,9995,8895,24326,15576,17627,13660
1,Argentina,30,4418724,9318,14459,28090,31429,24095,18687,47067,...,10547,15247,17202,9496,8983,20847,11921,21388,11473,12103
2,Australia,30,13977,0,0,0,0,25,0,0,...,5,0,0,0,5,0,0,0,0,0
3,Bangladesh,30,101114,619,266,347,306,677,369,240,...,205,345,414,358,387,459,308,307,743,467
4,Belize,30,1165487,5570,2993,2108,3206,1899,4140,3632,...,6606,11511,6616,4781,8772,16087,4560,4033,11667,21137


🧩 STEP 3 — Transform “Country primary loss” to Long Format
🧠 Why this step matters

- The raw data still has columns like tc_loss_ha_2002, tc_loss_ha_2003, etc.
- We need to convert these into a single “year” column for easier merging, plotting, and analysis later.

In [18]:
def melt_yearly_columns(df, prefix, value_name):
    """
    Converts wide year columns (e.g., tc_loss_ha_2002, tc_loss_ha_2003, …)
    into a tidy long format with columns: country, threshold, year, <value_name>.
    """
    year_cols = [c for c in df.columns if c.startswith(prefix)]
    if not year_cols:
        print(f"No columns found with prefix '{prefix}'. Check column names.")
        return df

    melted = df.melt(
        id_vars=[c for c in df.columns if c not in year_cols],
        value_vars=year_cols,
        var_name="metric_year",
        value_name=value_name
    )

    melted["year"] = melted["metric_year"].str.extract(r"(\d{4})").astype(int)
    melted = melted.drop(columns=["metric_year"])
    return melted

# --- Apply transformation ---
pl_tidy = melt_yearly_columns(pl_clean, prefix="tc_loss_ha_", value_name="primary_forest_loss_ha")

print("✅ Transformed shape:", pl_tidy.shape)
print("Columns:", list(pl_tidy.columns)[:10])

print("\n📊 Preview of tidy data (after transformation):")
display(pl_tidy.head(10))


✅ Transformed shape: (1748, 5)
Columns: ['country', 'threshold', 'area_ha', 'primary_forest_loss_ha', 'year']

📊 Preview of tidy data (after transformation):


,country,threshold,area_ha,primary_forest_loss_ha,year
0,Angola,30,2458061,3499,2002
1,Argentina,30,4418724,9318,2002
2,Australia,30,13977,0,2002
3,Bangladesh,30,101114,619,2002
4,Belize,30,1165487,5570,2002
5,Benin,30,1952,0,2002
6,Bhutan,30,1645545,119,2002
7,Bolivia,30,40850721,70494,2002
8,Brazil,30,343260979,1621738,2002
9,Brunei,30,431532,474,2002


💾 STEP 4 — Save Processed “Country primary loss” Data

In [19]:
import os

# Ensure folder exists
os.makedirs("../data/processed", exist_ok=True)

# Define output path
pl_out_path = "../data/processed/country_primary_loss_processed.csv"

# Save tidy dataset
pl_tidy.to_csv(pl_out_path, index=False)

print(f"💾 Saved processed dataset to: {pl_out_path}")
print(f"Rows: {len(pl_tidy):,} | Columns: {len(pl_tidy.columns)}")

# Verify save worked correctly
verify_pl = pd.read_csv(pl_out_path)
print("\n✅ Reloaded successfully! Sample below:")
display(verify_pl.head(5))


💾 Saved processed dataset to: ../data/processed/country_primary_loss_processed.csv
Rows: 1,748 | Columns: 5

✅ Reloaded successfully! Sample below:


,country,threshold,area_ha,primary_forest_loss_ha,year
0,Angola,30,2458061,3499,2002
1,Argentina,30,4418724,9318,2002
2,Australia,30,13977,0,2002
3,Bangladesh,30,101114,619,2002
4,Belize,30,1165487,5570,2002


🌾 3. Processing the “Country Drivers” Sheet

This dataset links annual tree cover loss to its dominant drivers, such as agriculture, logging, fires, or urbanization.
Unlike the previous sheets, it already contains a driver and year column, so our main tasks are to:

- Audit the data for structure, missing values, and unique driver types.
- Clean column names and handle duplicates or blanks.
- Pivot the driver column into multiple columns (one per driver) to enable country-level comparisons.
- Save the cleaned and pivoted version into data/processed/country_drivers_processed.csv.

🌾 STEP 1 — Audit “Country drivers”

🧠 Why this step matters

This sheet links forest loss to dominant drivers, allowing you to analyze which causes are most impactful globally or regionally.
Unlike the first two, this dataset already has a driver column (categorical) and a year column — but we’ll need to pivot it later to get one column per driver.

In [20]:
# --- Load raw 'Country drivers' sheet ---
drivers_raw = global_forest_watch_excel_file.parse("Country drivers")

print("✅ Loaded 'Country drivers' sheet")
print("Shape (rows, columns):", drivers_raw.shape)

# 1️⃣ Basic Info
print("\n🔹 Basic Info:")
drivers_raw.info()

# 2️⃣ Missing Values
print("\n🔹 Missing values per column:")
display(
    drivers_raw.isna()
    .sum()
    .to_frame("missing_count")
    .sort_values("missing_count", ascending=False)
    .head(10)
)

# 3️⃣ Numeric summary
print("\n🔹 Descriptive statistics (numeric columns):")
display(drivers_raw.describe(include=[np.number]).T.head(10))

# 4️⃣ Unique drivers
print("\n🔹 Unique driver categories:")
if "driver" in drivers_raw.columns:
    print(drivers_raw["driver"].unique())

# 5️⃣ Sample
print("\n🔹 Sample of raw data:")
display(drivers_raw.head(5))


✅ Loaded 'Country drivers' sheet
Shape (rows, columns): (21897, 5)

🔹 Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21897 entries, 0 to 21896
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     21897 non-null  object 
 1   threshold   21897 non-null  int64  
 2   driver      21897 non-null  object 
 3   year        21897 non-null  int64  
 4   tc_loss_ha  21897 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 855.5+ KB

🔹 Missing values per column:


,missing_count
country,0
threshold,0
driver,0
year,0
tc_loss_ha,0



🔹 Descriptive statistics (numeric columns):


,count,mean,std,min,25%,50%,75%,max
threshold,21897.0,30.000000,0.000000,30.0,30.0,30.0,30.0,30.0
year,21897.0,2012.406448,6.930328,2001.0,2006.0,2012.0,2018.0,2024.0
tc_loss_ha,21897.0,23513.404407,169882.759311,0.0,26.0,236.0,2375.0,7789588.0



🔹 Unique driver categories:
['Hard commodities' 'Logging' 'Other natural disturbances'
 'Permanent agriculture' 'Settlements & Infrastructure' 'Wildfire'
 'Shifting cultivation']

🔹 Sample of raw data:


,country,threshold,driver,year,tc_loss_ha
0,Afghanistan,30,Hard commodities,2014,0.0
1,Afghanistan,30,Logging,2001,3.0
2,Afghanistan,30,Logging,2002,64.0
3,Afghanistan,30,Logging,2003,73.0
4,Afghanistan,30,Logging,2004,143.0


🧩 STEP 2 — Cleaning “Country drivers”

In [21]:
# Work on a copy to keep raw data safe
drivers_clean = drivers_raw.copy()

# --- 1️⃣ Standardize column names ---
drivers_clean.columns = drivers_clean.columns.str.strip().str.lower().str.replace(" ", "_")

# --- 2️⃣ Clean country names ---
if "country" in drivers_clean.columns:
    drivers_clean["country"] = drivers_clean["country"].astype(str).str.strip().str.title()

# --- 3️⃣ Clean driver names ---
if "driver" in drivers_clean.columns:
    drivers_clean["driver"] = drivers_clean["driver"].astype(str).str.strip().str.title()

# --- 4️⃣ Replace empty strings with NaN ---
drivers_clean = drivers_clean.replace(r"^\s*$", pd.NA, regex=True)

# --- 5️⃣ Drop duplicates ---
before = len(drivers_clean)
drivers_clean = drivers_clean.drop_duplicates(subset=["country", "driver", "year", "threshold"], keep="first")
after = len(drivers_clean)
print(f"Removed {before - after} duplicate rows (if any).")

# --- 6️⃣ Convert numeric columns ---
for col in drivers_clean.columns:
    try:
        drivers_clean[col] = pd.to_numeric(drivers_clean[col])
    except (ValueError, TypeError):
        pass

# --- 7️⃣ Verify results ---
print("\n✅ After cleaning:")
display(drivers_clean.info())
display(drivers_clean.head(10))


Removed 0 duplicate rows (if any).

✅ After cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21897 entries, 0 to 21896
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     21897 non-null  object 
 1   threshold   21897 non-null  int64  
 2   driver      21897 non-null  object 
 3   year        21897 non-null  int64  
 4   tc_loss_ha  21897 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 855.5+ KB


None

,country,threshold,driver,year,tc_loss_ha
0,Afghanistan,30,Hard Commodities,2014,0.0
1,Afghanistan,30,Logging,2001,3.0
2,Afghanistan,30,Logging,2002,64.0
3,Afghanistan,30,Logging,2003,73.0
4,Afghanistan,30,Logging,2004,143.0
5,Afghanistan,30,Logging,2005,142.0
6,Afghanistan,30,Logging,2006,102.0
7,Afghanistan,30,Logging,2007,182.0
8,Afghanistan,30,Logging,2008,67.0
9,Afghanistan,30,Logging,2009,33.0


🔄 STEP 3 — Pivot “Country drivers”
🧠 Why pivot?

- Currently, each row represents one (country, driver, year) pair.
- We want to convert it into a wide format
- That way, each country–year is a single record with hectares of loss per driver.

In [22]:
import re

# --- Pivot drivers to wide format ---
drivers_pivot = (
    drivers_clean
    .pivot_table(
        index=["country", "threshold", "year"],
        columns="driver",
        values="tc_loss_ha",
        aggfunc="sum",
        fill_value=0
    )
    .reset_index()
)

# --- Clean column names (make them lowercase, replace spaces/special chars with underscores) ---
drivers_pivot.columns = [
    re.sub(r"[^0-9A-Za-z_]+", "_", str(c)).lower().strip("_")
    for c in drivers_pivot.columns
]

print("✅ Pivoted shape:", drivers_pivot.shape)
print("Columns:", drivers_pivot.columns[:10].tolist())

print("\n📊 Preview of pivoted data:")
display(drivers_pivot.head(10))


✅ Pivoted shape: (3625, 10)
Columns: ['country', 'threshold', 'year', 'hard_commodities', 'logging', 'other_natural_disturbances', 'permanent_agriculture', 'settlements_infrastructure', 'shifting_cultivation', 'wildfire']

📊 Preview of pivoted data:


,country,threshold,year,hard_commodities,logging,other_natural_disturbances,permanent_agriculture,settlements_infrastructure,shifting_cultivation,wildfire
0,Afghanistan,30,2001,0.0,3.0,2.0,63.0,0.0,0.0,1.0
1,Afghanistan,30,2002,0.0,64.0,3.0,49.0,0.0,0.0,34.0
2,Afghanistan,30,2003,0.0,73.0,1.0,11.0,0.0,0.0,134.0
3,Afghanistan,30,2004,0.0,143.0,1.0,24.0,0.0,0.0,13.0
4,Afghanistan,30,2005,0.0,142.0,4.0,12.0,0.0,0.0,51.0
5,Afghanistan,30,2006,0.0,102.0,0.0,10.0,0.0,0.0,23.0
6,Afghanistan,30,2007,0.0,182.0,0.0,9.0,0.0,0.0,36.0
7,Afghanistan,30,2008,0.0,67.0,2.0,7.0,0.0,0.0,19.0
8,Afghanistan,30,2009,0.0,33.0,0.0,8.0,0.0,0.0,12.0
9,Afghanistan,30,2010,0.0,67.0,0.0,4.0,0.0,0.0,6.0


💾 STEP 4 — Save Processed “Country drivers” Data

In [23]:
import os

# Ensure processed folder exists
os.makedirs("../data/processed", exist_ok=True)

# Define output path
drivers_out_path = "../data/processed/country_drivers_processed.csv"

# Save the pivoted (wide) dataset
drivers_pivot.to_csv(drivers_out_path, index=False)

print(f"💾 Saved processed dataset to: {drivers_out_path}")
print(f"Rows: {len(drivers_pivot):,} | Columns: {len(drivers_pivot.columns)}")

# Quick verification: reload to confirm structure
verify_drivers = pd.read_csv(drivers_out_path)
print("\n✅ Reloaded successfully! Sample below:")
display(verify_drivers.head(5))

💾 Saved processed dataset to: ../data/processed/country_drivers_processed.csv
Rows: 3,625 | Columns: 10

✅ Reloaded successfully! Sample below:


,country,threshold,year,hard_commodities,logging,other_natural_disturbances,permanent_agriculture,settlements_infrastructure,shifting_cultivation,wildfire
0,Afghanistan,30,2001,0.0,3.0,2.0,63.0,0.0,0.0,1.0
1,Afghanistan,30,2002,0.0,64.0,3.0,49.0,0.0,0.0,34.0
2,Afghanistan,30,2003,0.0,73.0,1.0,11.0,0.0,0.0,134.0
3,Afghanistan,30,2004,0.0,143.0,1.0,24.0,0.0,0.0,13.0
4,Afghanistan,30,2005,0.0,142.0,4.0,12.0,0.0,0.0,51.0


### 🌬️ 4. Processing the “Country carbon data” Sheet
Why this step is important

This dataset contains information on forest-related carbon fluxes — including gross emissions, carbon removals, and net GHG balance per country and year.
Analyzing these values allows us to quantify the climate impact of deforestation and restoration activities, directly linking forest dynamics to SDG 15 (Life on Land) and SDG 13 (Climate Action).

Because the raw sheet stores each year as a separate column (e.g.,
gfw_forest_carbon_gross_emissions_2001__Mg_CO2e, ..._2002__Mg_CO2e, etc.),
we must first audit the structure, verify column patterns, and check for issues such as missing values or inconsistent thresholds before reshaping it into a tidy format.

STEP 1- Audit raw data

In [24]:
# --- Load the raw sheet ---
carbon_raw = global_forest_watch_excel_file.parse("Country carbon data")

print("✅ Loaded 'Country carbon data' sheet")
print("Shape (rows, columns):", carbon_raw.shape)

# 1️⃣ Basic info
print("\n🔹 Basic Info:")
carbon_raw.info()

# 2️⃣ Missing values
print("\n🔹 Missing values per column:")
display(
    carbon_raw.isna()
    .sum()
    .to_frame("missing_count")
    .sort_values("missing_count", ascending=False)
    .head(10)
)

# 3️⃣ Numeric summary
print("\n🔹 Descriptive statistics (numeric columns):")
display(carbon_raw.describe(include=[np.number]).T.head(10))

# 4️⃣ Check for any threshold or country naming issues
print("\n🔹 Unique threshold values (if available):")
if "umd_tree_cover_density_2000__threshold" in carbon_raw.columns:
    print(carbon_raw["umd_tree_cover_density_2000__threshold"].unique())

print("\n🔹 Sample preview:")
display(carbon_raw.head(5))


✅ Loaded 'Country carbon data' sheet
Shape (rows, columns): (498, 32)

🔹 Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 32 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   country                                            498 non-null    object 
 1   umd_tree_cover_density_2000__threshold             498 non-null    int64  
 2   umd_tree_cover_extent_2000__ha                     498 non-null    int64  
 3   gfw_aboveground_carbon_stocks_2000__Mg_C           498 non-null    int64  
 4   avg_gfw_aboveground_carbon_stocks_2000__Mg_C_ha-1  498 non-null    int64  
 5   gfw_forest_carbon_gross_emissions__Mg_CO2e_yr-1    498 non-null    int64  
 6   gfw_forest_carbon_gross_removals__Mg_CO2_yr-1      498 non-null    int64  
 7   gfw_forest_carbon_net_flux__Mg_CO2e_yr-1           498 non-null    int64  
 8   gfw_f

,missing_count
country,0
umd_tree_cover_density_2000__threshold,0
gfw_forest_carbon_gross_emissions_2023__Mg_CO2e,0
gfw_forest_carbon_gross_emissions_2022__Mg_CO2e,0
gfw_forest_carbon_gross_emissions_2021__Mg_CO2e,0
gfw_forest_carbon_gross_emissions_2020__Mg_CO2e,0
gfw_forest_carbon_gross_emissions_2019__Mg_CO2e,0
gfw_forest_carbon_gross_emissions_2018__Mg_CO2e,0
gfw_forest_carbon_gross_emissions_2017__Mg_CO2e,0
gfw_forest_carbon_gross_emissions_2016__Mg_CO2e,0



🔹 Descriptive statistics (numeric columns):


,count,mean,std,min,25%,50%,75%,max
umd_tree_cover_density_2000__threshold,498.0,5.166667e+01,1.842745e+01,3.000000e+01,30.00,50.0,7.500000e+01,7.500000e+01
umd_tree_cover_extent_2000__ha,498.0,1.931269e+07,6.887766e+07,0.000000e+00,142635.25,2085215.5,9.780736e+06,7.612265e+08
gfw_aboveground_carbon_stocks_2000__Mg_C,498.0,1.587066e+09,5.357827e+09,0.000000e+00,9929741.50,156212670.5,6.875177e+08,5.556887e+10
avg_gfw_aboveground_carbon_stocks_2000__Mg_C_ha-1,498.0,3.367570e+02,1.781686e+02,0.000000e+00,210.00,320.0,4.655000e+02,7.700000e+02
gfw_forest_carbon_gross_emissions__Mg_CO2e_yr-1,498.0,4.839584e+07,1.696282e+08,0.000000e+00,193746.25,3567744.0,2.802821e+07,1.589982e+09
gfw_forest_carbon_gross_removals__Mg_CO2_yr-1,498.0,7.027193e+07,2.189253e+08,0.000000e+00,1035810.50,11793397.5,4.982675e+07,1.998131e+09
gfw_forest_carbon_net_flux__Mg_CO2e_yr-1,498.0,-2.187608e+07,1.044479e+08,-1.373904e+09,-19619187.75,-3203279.5,-5.210875e+04,3.995718e+08
gfw_forest_carbon_gross_emissions_2001__Mg_CO2e,498.0,2.762562e+07,1.143494e+08,0.000000e+00,122655.50,2095027.0,1.264239e+07,1.221139e+09
gfw_forest_carbon_gross_emissions_2002__Mg_CO2e,498.0,3.408261e+07,1.531540e+08,0.000000e+00,93762.50,1902934.0,1.246289e+07,1.655445e+09
gfw_forest_carbon_gross_emissions_2003__Mg_CO2e,498.0,2.824519e+07,1.306350e+08,0.000000e+00,101137.25,1336894.5,9.653048e+06,1.514613e+09



🔹 Unique threshold values (if available):
[30 50 75]

🔹 Sample preview:


,country,umd_tree_cover_density_2000__threshold,umd_tree_cover_extent_2000__ha,gfw_aboveground_carbon_stocks_2000__Mg_C,avg_gfw_aboveground_carbon_stocks_2000__Mg_C_ha-1,gfw_forest_carbon_gross_emissions__Mg_CO2e_yr-1,gfw_forest_carbon_gross_removals__Mg_CO2_yr-1,gfw_forest_carbon_net_flux__Mg_CO2e_yr-1,gfw_forest_carbon_gross_emissions_2001__Mg_CO2e,gfw_forest_carbon_gross_emissions_2002__Mg_CO2e,...,gfw_forest_carbon_gross_emissions_2015__Mg_CO2e,gfw_forest_carbon_gross_emissions_2016__Mg_CO2e,gfw_forest_carbon_gross_emissions_2017__Mg_CO2e,gfw_forest_carbon_gross_emissions_2018__Mg_CO2e,gfw_forest_carbon_gross_emissions_2019__Mg_CO2e,gfw_forest_carbon_gross_emissions_2020__Mg_CO2e,gfw_forest_carbon_gross_emissions_2021__Mg_CO2e,gfw_forest_carbon_gross_emissions_2022__Mg_CO2e,gfw_forest_carbon_gross_emissions_2023__Mg_CO2e,gfw_forest_carbon_gross_emissions_2024__Mg_CO2e
0,Afghanistan,30,205771,12409398,123,15339,376800,-361461,27986.0,41762.0,...,0.0,0.0,0.0,4893.0,3708.0,11409.0,6772.0,1913.0,3435.0,2636.0
1,Afghanistan,50,148417,9765465,134,12657,275855,-263199,25603.0,32691.0,...,0.0,0.0,0.0,3920.0,3343.0,10321.0,6045.0,1664.0,2530.0,2106.0
2,Afghanistan,75,75480,5571655,150,6147,151074,-144926,15780.0,15308.0,...,0.0,0.0,0.0,1962.0,1743.0,6451.0,2477.0,668.0,1857.0,1512.0
3,Albania,30,648459,40958831,238,721806,5103589,-4381783,1417747.0,348556.0,...,120041.0,334094.0,448993.0,724335.0,429556.0,427420.0,506228.0,649874.0,948758.0,308121.0
4,Albania,50,534671,37239867,263,682919,4294627,-3611709,1358272.0,338279.0,...,113553.0,304691.0,403366.0,669011.0,404887.0,391385.0,449937.0,591504.0,895138.0,275104.0


🌬️ STEP 2 — Cleaning the “Country carbon data” Sheet

In [25]:
# Work on a copy to keep raw safe
carbon_clean = carbon_raw.copy()

# --- 1️⃣ Standardize column names ---
carbon_clean.columns = carbon_clean.columns.str.strip().str.lower().str.replace(" ", "_")

# --- 2️⃣ Rename threshold column (for consistency) ---
if "umd_tree_cover_density_2000__threshold" in carbon_clean.columns:
    carbon_clean = carbon_clean.rename(columns={"umd_tree_cover_density_2000__threshold": "threshold"})
    print("Renamed 'umd_tree_cover_density_2000__threshold' → 'threshold'")

# --- 3️⃣ Clean country names ---
if "country" in carbon_clean.columns:
    carbon_clean["country"] = carbon_clean["country"].astype(str).str.strip().str.title()

# --- 4️⃣ Replace empty strings with NaN ---
carbon_clean = carbon_clean.replace(r"^\s*$", pd.NA, regex=True)

# --- 5️⃣ Drop duplicates ---
before = len(carbon_clean)
carbon_clean = carbon_clean.drop_duplicates(subset=["country", "threshold"], keep="first")
after = len(carbon_clean)
print(f"Removed {before - after} duplicate rows (if any).")

# --- 6️⃣ Convert numeric-like columns ---
for col in carbon_clean.columns:
    try:
        carbon_clean[col] = pd.to_numeric(carbon_clean[col])
    except (ValueError, TypeError):
        pass

# --- 7️⃣ Verify results ---
print("\n✅ After cleaning:")
display(carbon_clean.info())
display(carbon_clean.head(5))


Renamed 'umd_tree_cover_density_2000__threshold' → 'threshold'
Removed 0 duplicate rows (if any).

✅ After cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 32 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   country                                            498 non-null    object 
 1   threshold                                          498 non-null    int64  
 2   umd_tree_cover_extent_2000__ha                     498 non-null    int64  
 3   gfw_aboveground_carbon_stocks_2000__mg_c           498 non-null    int64  
 4   avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1  498 non-null    int64  
 5   gfw_forest_carbon_gross_emissions__mg_co2e_yr-1    498 non-null    int64  
 6   gfw_forest_carbon_gross_removals__mg_co2_yr-1      498 non-null    int64  
 7   gfw_forest_carbon_net_flux__mg_co2e_yr-1           49

None

,country,threshold,umd_tree_cover_extent_2000__ha,gfw_aboveground_carbon_stocks_2000__mg_c,avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1,gfw_forest_carbon_gross_emissions__mg_co2e_yr-1,gfw_forest_carbon_gross_removals__mg_co2_yr-1,gfw_forest_carbon_net_flux__mg_co2e_yr-1,gfw_forest_carbon_gross_emissions_2001__mg_co2e,gfw_forest_carbon_gross_emissions_2002__mg_co2e,...,gfw_forest_carbon_gross_emissions_2015__mg_co2e,gfw_forest_carbon_gross_emissions_2016__mg_co2e,gfw_forest_carbon_gross_emissions_2017__mg_co2e,gfw_forest_carbon_gross_emissions_2018__mg_co2e,gfw_forest_carbon_gross_emissions_2019__mg_co2e,gfw_forest_carbon_gross_emissions_2020__mg_co2e,gfw_forest_carbon_gross_emissions_2021__mg_co2e,gfw_forest_carbon_gross_emissions_2022__mg_co2e,gfw_forest_carbon_gross_emissions_2023__mg_co2e,gfw_forest_carbon_gross_emissions_2024__mg_co2e
0,Afghanistan,30,205771,12409398,123,15339,376800,-361461,27986.0,41762.0,...,0.0,0.0,0.0,4893.0,3708.0,11409.0,6772.0,1913.0,3435.0,2636.0
1,Afghanistan,50,148417,9765465,134,12657,275855,-263199,25603.0,32691.0,...,0.0,0.0,0.0,3920.0,3343.0,10321.0,6045.0,1664.0,2530.0,2106.0
2,Afghanistan,75,75480,5571655,150,6147,151074,-144926,15780.0,15308.0,...,0.0,0.0,0.0,1962.0,1743.0,6451.0,2477.0,668.0,1857.0,1512.0
3,Albania,30,648459,40958831,238,721806,5103589,-4381783,1417747.0,348556.0,...,120041.0,334094.0,448993.0,724335.0,429556.0,427420.0,506228.0,649874.0,948758.0,308121.0
4,Albania,50,534671,37239867,263,682919,4294627,-3611709,1358272.0,338279.0,...,113553.0,304691.0,403366.0,669011.0,404887.0,391385.0,449937.0,591504.0,895138.0,275104.0


🌬️ STEP 3 — Transform “Country carbon data” to Long Format

🧠 Why this matters

Right now, each year’s emission data is a separate column.
By melting them into one year column and one carbon_gross_emissions_MgCO2e column, you’ll:

- Align this dataset with all others (same tidy format).
- Make it mergeable and easy to visualize.
- Enable time-series and regression analysis later.

In [27]:
import re

def melt_carbon_columns(df, pattern, value_name):
    """
    Melts all columns matching a yearly carbon emission pattern into a tidy long format.
    Example: gfw_forest_carbon_gross_emissions_2001__Mg_CO2e → year: 2001, value.
    """
    # Detect all year columns using regex pattern
    year_cols = [c for c in df.columns if re.match(pattern, c)]
    if not year_cols:
        print("❌ No matching year columns found. Check column names.")
        return df

    melted = df.melt(
        id_vars=[c for c in df.columns if c not in year_cols],
        value_vars=year_cols,
        var_name="metric_year",
        value_name=value_name
    )

    # Extract year from column names
    melted["year"] = melted["metric_year"].str.extract(r"(\d{4})").astype(int)
    melted = melted.drop(columns=["metric_year"])

    return melted


# --- Apply transformation ---
carbon_tidy = melt_carbon_columns(
    carbon_clean,
    pattern=r"^gfw_forest_carbon_gross_emissions_\d{4}__mg_co2e$",
    value_name="carbon_gross_emissions_MgCO2e"
)

print("✅ Transformed shape:", carbon_tidy.shape)
print("Columns:", list(carbon_tidy.columns)[:10])

print("\n📊 Preview of tidy data (after transformation):")
display(carbon_tidy.head(10))


✅ Transformed shape: (11952, 10)
Columns: ['country', 'threshold', 'umd_tree_cover_extent_2000__ha', 'gfw_aboveground_carbon_stocks_2000__mg_c', 'avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1', 'gfw_forest_carbon_gross_emissions__mg_co2e_yr-1', 'gfw_forest_carbon_gross_removals__mg_co2_yr-1', 'gfw_forest_carbon_net_flux__mg_co2e_yr-1', 'carbon_gross_emissions_MgCO2e', 'year']

📊 Preview of tidy data (after transformation):


,country,threshold,umd_tree_cover_extent_2000__ha,gfw_aboveground_carbon_stocks_2000__mg_c,avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1,gfw_forest_carbon_gross_emissions__mg_co2e_yr-1,gfw_forest_carbon_gross_removals__mg_co2_yr-1,gfw_forest_carbon_net_flux__mg_co2e_yr-1,carbon_gross_emissions_MgCO2e,year
0,Afghanistan,30,205771,12409398,123,15339,376800,-361461,27986.0,2001
1,Afghanistan,50,148417,9765465,134,12657,275855,-263199,25603.0,2001
2,Afghanistan,75,75480,5571655,150,6147,151074,-144926,15780.0,2001
3,Albania,30,648459,40958831,238,721806,5103589,-4381783,1417747.0,2001
4,Albania,50,534671,37239867,263,682919,4294627,-3611709,1358272.0,2001
5,Albania,75,363706,28761196,298,576299,3001723,-2425424,1137609.0,2001
6,Algeria,30,1223325,64822106,313,1872312,4873094,-3000781,574332.0,2001
7,Algeria,50,895366,50658903,334,1540229,3547408,-2007180,444098.0,2001
8,Algeria,75,496534,31035068,366,952542,1988182,-1035640,250927.0,2001
9,Angola,30,55276135,2879806419,296,62402574,170616018,-108213442,39294740.0,2001


💾 STEP 4 — Save Processed “Country carbon data”

In [28]:
import os

# Ensure processed folder exists
os.makedirs("../data/processed", exist_ok=True)

# Define output path
carbon_out_path = "../data/processed/country_carbon_processed.csv"

# Save tidy dataset
carbon_tidy.to_csv(carbon_out_path, index=False)

print(f"💾 Saved processed dataset to: {carbon_out_path}")
print(f"Rows: {len(carbon_tidy):,} | Columns: {len(carbon_tidy.columns)}")

# Verify save worked correctly
verify_carbon = pd.read_csv(carbon_out_path)
print("\n✅ Reloaded successfully! Sample below:")
display(verify_carbon.head(5))


💾 Saved processed dataset to: ../data/processed/country_carbon_processed.csv
Rows: 11,952 | Columns: 10

✅ Reloaded successfully! Sample below:


,country,threshold,umd_tree_cover_extent_2000__ha,gfw_aboveground_carbon_stocks_2000__mg_c,avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1,gfw_forest_carbon_gross_emissions__mg_co2e_yr-1,gfw_forest_carbon_gross_removals__mg_co2_yr-1,gfw_forest_carbon_net_flux__mg_co2e_yr-1,carbon_gross_emissions_MgCO2e,year
0,Afghanistan,30,205771,12409398,123,15339,376800,-361461,27986.0,2001
1,Afghanistan,50,148417,9765465,134,12657,275855,-263199,25603.0,2001
2,Afghanistan,75,75480,5571655,150,6147,151074,-144926,15780.0,2001
3,Albania,30,648459,40958831,238,721806,5103589,-4381783,1417747.0,2001
4,Albania,50,534671,37239867,263,682919,4294627,-3611709,1358272.0,2001


🌍 Integrating All Country-Level Datasets
Why this step is important

Up to this point, we have individually cleaned and reshaped four key country-level datasets from Global Forest Watch:

- Tree Cover Loss – annual loss of forested area
- Primary Forest Loss – loss in humid tropical primary forests
- Drivers of Deforestation – hectares of loss by cause (fire, agriculture, etc.)
- Carbon Data – annual forest-related CO₂ emissions and removals

Each dataset provides a different perspective on global forest change.
To perform meaningful Exploratory Data Analysis (EDA), predictive modeling, and visualization, we now need a single integrated dataset that combines all relevant variables per country and year.

This section merges the four processed datasets on their common identifiers —
country, threshold, and year — ensuring that all information aligns correctly in one master table.
Missing values will be preserved (NaN) so that no data is lost during integration.

🧩 STEP 1 — Merge All Country-Level Datasets

In [29]:
import pandas as pd
import os

# Load processed files
base_path = "../data/processed"

tcl = pd.read_csv(f"{base_path}/country_tree_cover_loss_processed.csv")
pl = pd.read_csv(f"{base_path}/country_primary_loss_processed.csv")
drv = pd.read_csv(f"{base_path}/country_drivers_processed.csv")
crb = pd.read_csv(f"{base_path}/country_carbon_processed.csv")

print("✅ Loaded all processed datasets:")
for name, df in zip(["Tree Cover Loss", "Primary Loss", "Drivers", "Carbon"], [tcl, pl, drv, crb]):
    print(f"{name:<15}: {df.shape}")

# --- Merge progressively on country + year (keeping all thresholds if present) ---
merged = (
    tcl
    .merge(pl, on=["country", "threshold", "year"], how="outer")
    .merge(drv, on=["country", "threshold", "year"], how="outer")
    .merge(crb, on=["country", "threshold", "year"], how="outer")
)

print("\n✅ Merged dataset shape:", merged.shape)
print("Columns:", merged.columns[:12].tolist(), "...")
display(merged.head(10))


✅ Loaded all processed datasets:
Tree Cover Loss: (31872, 8)
Primary Loss   : (1748, 5)
Drivers        : (3625, 10)
Carbon         : (11952, 10)

✅ Merged dataset shape: (31873, 24)
Columns: ['country', 'threshold', 'area_ha_x', 'extent_2000_ha', 'extent_2010_ha', 'gain_2000-2012_ha', 'tree_cover_loss_ha', 'year', 'area_ha_y', 'primary_forest_loss_ha', 'hard_commodities', 'logging'] ...


,country,threshold,area_ha_x,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tree_cover_loss_ha,year,area_ha_y,primary_forest_loss_ha,...,settlements_infrastructure,shifting_cultivation,wildfire,umd_tree_cover_extent_2000__ha,gfw_aboveground_carbon_stocks_2000__mg_c,avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1,gfw_forest_carbon_gross_emissions__mg_co2e_yr-1,gfw_forest_carbon_gross_removals__mg_co2_yr-1,gfw_forest_carbon_net_flux__mg_co2e_yr-1,carbon_gross_emissions_MgCO2e
0,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,103.0,2001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,214.0,2002,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,267.0,2003,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,226.0,2004,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,268.0,2005,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,172.0,2006,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,274.0,2007,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,123.0,2008,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,92.0,2009,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,109.0,2010,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


🧩 STEP 2 — Evaluate Missing Values

In [30]:
# --- 1️⃣ Overall missingness percentage ---
missing_summary = (
    merged.isna()
    .mean()
    .sort_values(ascending=False)
    .to_frame("missing_ratio")
)
display(missing_summary.head(15))

# --- 2️⃣ Country-level missing summary ---
country_missing = merged.groupby("country").apply(lambda x: x.isna().mean().mean())
print("\n🔹 Average missingness by country (first 10):")
display(country_missing.sort_values(ascending=False).head(10))

# --- 3️⃣ Check if certain columns are systematically missing ---
cols_all_missing = [c for c in merged.columns if merged[c].isna().all()]
if cols_all_missing:
    print("\n⚠️ Columns completely empty:", cols_all_missing)
else:
    print("\n✅ No completely empty columns detected.")


,missing_ratio
area_ha_y,0.945157
primary_forest_loss_ha,0.945157
other_natural_disturbances,0.886267
shifting_cultivation,0.886267
hard_commodities,0.886267
logging,0.886267
wildfire,0.886267
permanent_agriculture,0.886267
settlements_infrastructure,0.886267
gfw_forest_carbon_net_flux__mg_co2e_yr-1,0.625012



🔹 Average missingness by country (first 10):


/var/folders/3d/0t0vhvyj2m30_zwy784tbmdr0000gn/T/ipykernel_58156/2462634743.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  country_missing = merged.groupby("country").apply(lambda x: x.isna().mean().mean())


country
S√£O Tom√© And Pr√≠Ncipe    0.583333
Saint-Barthélemy            0.557292
Iceland                     0.557292
Faroe Islands               0.557292
São Tomé And Príncipe       0.557292
Saudi Arabia                0.557292
Oman                        0.557292
Djibouti                    0.557292
United Arab Emirates        0.557292
Yemen                       0.557292
dtype: float64


✅ No completely empty columns detected.


💾 STEP 3 — Save Final Merged Dataset

In [34]:
# Ensure processed folder exists
import os
os.makedirs("../data/processed", exist_ok=True)

# Define output path
merged_out_path = "../data/processed/merged_clean_data.csv"

# Save merged dataset
merged.to_csv(merged_out_path, index=False)

print(f"💾 Final merged dataset saved to: {merged_out_path}")
print(f"Rows: {len(merged):,} | Columns: {len(merged.columns)}")

# Quick verification
verify_merged = pd.read_csv(merged_out_path)
print("\n✅ Reloaded successfully! Sample below:")
display(verify_merged.head(10))


💾 Final merged dataset saved to: ../data/processed/merged_clean_data.csv
Rows: 31,873 | Columns: 24

✅ Reloaded successfully! Sample below:


,country,threshold,area_ha_x,extent_2000_ha,extent_2010_ha,gain_2000-2012_ha,tree_cover_loss_ha,year,area_ha_y,primary_forest_loss_ha,...,settlements_infrastructure,shifting_cultivation,wildfire,umd_tree_cover_extent_2000__ha,gfw_aboveground_carbon_stocks_2000__mg_c,avg_gfw_aboveground_carbon_stocks_2000__mg_c_ha-1,gfw_forest_carbon_gross_emissions__mg_co2e_yr-1,gfw_forest_carbon_gross_removals__mg_co2_yr-1,gfw_forest_carbon_net_flux__mg_co2e_yr-1,carbon_gross_emissions_MgCO2e
0,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,103.0,2001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,214.0,2002,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,267.0,2003,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,226.0,2004,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,268.0,2005,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,172.0,2006,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,274.0,2007,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,123.0,2008,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,92.0,2009,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Afghanistan,0,64383655.0,64383655.0,64383655.0,10738.0,109.0,2010,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
